# 6.天気予報のWebサイトをスクレイピングする

tenki.jp( https://tenki.jp/ )をスクレイピングする。

In [ ]:
# ライブラリをインポート
from urllib.request import urlopen # Webページ読み込み用関数
from bs4 import BeautifulSoup        # スクレイピングライブラリ

In [ ]:
# 東京都(千代田区)の天気予報を読み込む
r = urlopen("https://tenki.jp/forecast/3/16/4410/13101/")

In [ ]:
bhtml = r.read()   # HTMLを文字列(バイト文字列)として読み込む

In [ ]:
# エンコードを指定してバイト型を文字列型に変換
html = bhtml.decode("utf-8")

## soupオブジェクトを使う

In [ ]:
# スクレイピング用オブジェクト(soupオブジェクト)を作る
soup = BeautifulSoup(html, "html.parser")

In [ ]:
# soupオブジェクトを使って，タグ(エレメント)を検索する
# find_all()メソッドを使う。引数に，タグ名(div)を文字列として渡す
divs = soup.find_all("div")

In [ ]:
# divsには，HTML内のdivタグがすべて返ってくる
# divタグは複数あるので，リストとして返ってくる

len(divs)   # divタグの数を数えてみる

In [ ]:
divs[24]   # 0から数えて24番目のdivタグを表示してみる(文字列が表示される)

In [ ]:
navdiv = divs[24] # 実は検索結果もsoupオブジェクト

In [ ]:
navdiv.find_all('dd') # 検索結果を取り出し，さらにその子供だけを検索できる

## 「属性」と「属性値」を使った検索

HTMLの属性とは，「＜div id="foo">」の「id="foo"」の部分や，「＜span class="bar">」の「class="bar"」のようなタグ内部の部分。

「id」や「class」を「属性名」と呼び，「="~"」の引用符の中を「属性値」と呼ぶ。

たとえば，先ほど取り出したdivタグには，「class="weather-wrap clearfix"」という属性と属性値がある。この文字列を使って要素を検索できる。

In [ ]:
# HTML全体を指すsoupオブジェクトを使って検索

# タグ名div, classが"weather-wrap clearfix"の要素を検索する
# class_="weather-wrap clearfix"というように引数を指定するのがポイント
# ※検索時にclass属性を指定する時は，「class_="..."」とする(「class=".."」ではないので注意)

navdiv = soup.find_all("div", class_="weather-wrap clearfix") 
navdiv  # 先ほど取り出したゼロから数えて24番目のdivと同じ要素が返ってくる

## テキストの取り出し

普通，タグにはスクレイピングに必要な情報は入っていない。「<span>テキスト</span>」のようにタグに囲まれた部分に，必要な文字列が入っている。

In [ ]:
today_l = soup.find_all("div", class_="weather-wrap clearfix")  # 今日の天気を含むdiv要素を取り出す
# リストが返ってくるので，インデックスを指定して一つの要素を取り出す
today = today_l[0]
today  # 表示

In [ ]:
today.text   # タグに囲まれたテキスト部分だけを取り出す

In [ ]:
# 天気予報のテキストだけが欲しいので，さらに絞り込む
# 天気は"weather-telop"というclass属性に入っている
weather_l = today.find_all("p", class_="weather-telop")
weather = weather_l[0]  # リストが返ってくるのでインデックスで最初の要素を取り出す
weather   # 表示

In [ ]:
weatherstr = weather.text  # タグで囲まれたテキストを取り出す
weatherstr   # 表示

## スクレイピングのベストプラクティス

### 一発で取り出す

取り出したいテキストを含むタグを一発で取り出せるよう，タグ名やid, class_などを使ってfind_all()で取り出せるか考える。

### 絞り込んで取り出す

取り出したいテキストの親にあたるタグをfind_all()で取り出し，さらに絞り込んで取り出す。

## ライブコーディングでスクレイピングをするプログラムを書いてみる

### 目標1

東京の今日の天気スクレイピングで取り出す。

「今日の天気は~」という文字列を組み立てる。

### 目標2

- 明日の天気も取り出し「明日の天気は~」という文字列を追加する

- できた文字列をgttsで喋らせる

### 目標3

課題1を参考に，エキサイト天気予報のURLを文字列で受け取り，今日の天気を文字列として組み立てる関数を作る。関数名は「get_weather」とする。

関数から「return 変数」のようにして文字列を「戻り値」として戻す。

関数に別の地域のURLを渡して，天気予報の文字列を得られるようにする。

### 目標4

- 明日の天気も取り出し「明日の天気は~」という文字列を追加する

- 得た文字列をgttsで喋らせる。

- いろいろな地域の天気を喋らせてみる。

- 地域の名前(「東京都 伊豆諸島北部(大島)」など)を取り出し，関数で組み立てる文字列の先頭に追加する。

In [ ]:
# URL文字列を変数に代入
url = "https://tenki.jp/forecast/3/16/4410/13101/"

# 東京の天気予報を読み込む
r = urlopen(url)
bhtml = r.read()   # HTMLを文字列(バイト文字列)として読み込む
# エンコードを指定してバイト型を文字列型に変換
html = bhtml.decode("utf-8")

# スクレイピング用オブジェクト(soupオブジェクト)を作る
soup = BeautifulSoup(html, "html.parser")

# 今日の天気を含むdiv要素を取り出す
today_l = soup.find_all("div", class_="weather-wrap clearfix")
# リストが返ってくるので，インデックスを指定して一つの要素を取り出す
today = today_l[0]

weather_l = today.find_all("p", class_="weather-telop")
weather = weather_l[0]  # リストが返ってくるのでインデックスで最初の要素を取り出す
weatherstr = weather.text  # タグで囲まれたテキストを取り出す

In [ ]:
weatherstr